In [21]:
# Importing required modules
import xpress as xp
import networkx as nx

# Required inputs into the PC-TSP Algorithm:
#      1- List of donor-receipient pairs
#      2- List of altruisitic donors
#      3- List of nodes = donor-receipient pairs + altruisitic donors
#      4- Dictionary of edges with key ("node1","node2") and value is the weight of the edge
#      5- Dictionary of cycles with key (edge1,edge2,edge3,...) and value is the weight of the cycle i.e., 
#         sum of edges in the cycle up to and including length k


# Example
pairs = ["P1", "P2", "P3"] 
altruistic_donors = ["NDD1"]
nodes = pairs + altruistic_donors
edges = {("NDD1", "P1"): 0.10, 
         ("NDD1", "P2"): 0.8,
         ("P1", "P3"): 8, 
         ("P2", "P3"): 9,
         ("P3", "P1"): 6,
         ("P1", "P2"): 6}
all_cycles = {("P1","P3"): 14}


# Create the graph outside the separation algorithm and feed it in whenever we use it so that it doesn't create it every time
# instead, it just updates the weights i.e., you only create the graph once!

# Construct the directed graph G = (V, E, w) based on supplementary information definition 
G = nx.DiGraph()
for node in nodes:
    G.add_node(node)

# Add super source 'SUPER' and connect to all NDDs
super_source = "SUPER"
G.add_node(super_source)
for u in altruistic_donors:
    G.add_edge(super_source, u, weight=1)

In [28]:
# This cell contains the callbacks and their implementation methods, it must be run before running the model
# Note: Most of the functions have print statements as a method of debugging once we run the model on the full
#       dataset as we anticipate to have some errors. These will be removed once we are done with it. 

def separation_algorithm(y, f_i, G=G):
    """
    Implements the separation algorithm for cut-set constraints based on the supplementary information.
    
    >>> Inputs:
    y:                    Dictionary where keys are edge tuples (u, v) and values are their solution values i.e., y[e] gotten via current_sol[0] from above
    f_i:                  Dictionary where keys are nodes and values are the solver solution for f_i[v] gotten via current_sol[2] from above
    G:                    Graph object from Networkx of the current problem.

    >>> Outputs:
    delta_minus_S:       List of edges i.e., (u,v) that are in the named set as per the paper
    v:                   Node at which the constraint is violated for
    """
    
    print("###################### EXECUTING SEPARATION ALGORITHM ###############################")
    
    # Add edges with weights from y (solution values)
    # Note: as per the documentation adding an edge that already exists updates the edge data.
    for (u,v), sol_value in y.items():
        G.add_edge(u, v, weight=sol_value)
        
    
    # Solve the max-flow min-cut problem for each node with f_i[v] > 0
    for v, flow_in in f_i.items():
        if flow_in > 0:
            
            # Compute min-cut using the function from networkx
            cut_value, (Not_S, S) = nx.minimum_cut(G, super_source, v, capacity="weight")
            
            # If the cut weight is less than flow_in, add a violated constraint
            if cut_value < flow_in:

                print("$$$$$$$$$$$$$$$$$$$$$$$$$ WE FOUND A VIOLATED CONSTRAINT $$$$$$$$$$$$$$")
                print(f"Violated constraint for node {v}: Cut separating {Not_S} and {S}")
 
                
                # Define delta_minus_S in this case
                delta_minus_S = [(u, v) for u, v in G.edges(Not_S) if v in S]

                print(delta_minus_S, v)
                # Return a list that contains the violated constraint's data needed to add the cut into 
                # Xpress using the call back.
                return(delta_minus_S, v)

    # Note: This version returns the first violated constraint found; A possible extension is 
    #       to try if its better to return all violated constraints in a single potential solution. 
    return()






def separation_cbpreintsol(prob, data, soltype, cutoff):
    """
    Implements the separation callback when an integer solution is found by heuristics
    or during the branch and bound search, but before it is accepted by the optimizer.
    
    >>> Inputs:
    prob:                    Xpress model object to be passed to the callback function.
    data:                    Data we want to pass into the callback. 
    soltype:                 Type of MIP solution that has been found from 0-3 
    cutoff:                  Current cutoff value for the solution.

    >>> Outputs:
    ifreject:                Binary; 1 if the solution should be rejected and 0 otherwise
    newcutoff:               New cutoff value, to be used by the optimizer if the solution is accepted. 

    Note: The inputs and outputs are required and predetermined in this structure based on Xpress. 
    """
    
    print("@@@@@@@@@@@@@@@@@@@@@@@@@@ cbpreintsol triggered @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

    # Initialisze outputs:
    ifreject = 0
    newcutoff = None # Note: this is not being used currently, we will potentially explore it in extensions
    
    # Get LP relaxation solution
    lp_solution = []
    prob.getlpsol(lp_solution, None, None, None)

    print(lp_solution)
    
    # Populate values from lp_solution using iteration which should be faster as it is just an O(1) process.
    # Note: we could stop at f_i_temp if it gets too slow. 
    solution_iter = iter(lp_solution)
    
    for key in y_temp:
        y_temp[key] = next(solution_iter)
    
    for key in z_temp:
        z_temp[key] = next(solution_iter)
    
    for key in f_i_temp:
        f_i_temp[key] = next(solution_iter)
    
    for key in f_o_temp:
        f_o_temp[key] = next(solution_iter)
    
    # Get the violated constraint data using the separation algorithm from above
    violated_constraint_data = separation_algorithm(y_temp, f_i_temp)

    # Check if there is a violated constraint
    if violated_constraint_data:

        # Reject the solution
        ifreject = 1
        
        print("------------------------------ Solution Rejected -------------------------------")

        # There is a violated constraint so return 1/True i.e., reject this solution
        # Note: From the research I've found the purpose of this callback is to just reject solutions
        #       and not to add constraints to the problem dynamically. This is particularly relevant
        #       for Andrew's part because he'll probably have to do it recursively outside the solver. 
            
    # Otherwise, return 0/False i.e., this solution does not violate a constraint so its good!
    return(ifreject, newcutoff)






def separation_cboptnode(prob, data):
    """
    Implements the separation callback when called during the branch and bound search, after the LP
    relaxation has been solved for the current node, and after any internal cuts and heuristics have been
    applied, but before the optimizer checks if the current node should be branched. 
        
    >>> Inputs:
    prob:                    Xpress model object to be passed to the callback function.
    data:                    Data we want to pass into the callback. 
    
    >>> Outputs:
    feas:                    Feasibility status; If set to a nonzero value the current node is set to infeasible

    Note: The inputs and outputs are required and predetermined in this structure based on Xpress. 
    """


    
    print("@@@@@@@@@@@@@@@@@@@@@@@@@@ cboptnode triggered @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

    # Initialise output 
    feas = 0
    
    # Get LP relaxation solution
    lp_solution = []
    prob.getlpsol(lp_solution, None, None, None)
    
    # Populate values from lp_solution using iteration which should be faster as it is just an O(1) process.
    # Note: we could stop at f_i_temp if it gets too slow. 
    solution_iter = iter(lp_solution)
    
    for key in y_temp:
        y_temp[key] = next(solution_iter)
    
    for key in z_temp:
        z_temp[key] = next(solution_iter)
    
    for key in f_i_temp:
        f_i_temp[key] = next(solution_iter)
    
    # for key in f_o_temp:
    #     f_o_temp[key] = next(solution_iter)

    
    # Get the violated constraint data using the separation algorithm from above
    violated_constraint_data = separation_algorithm(y_temp, f_i_temp)
    
    # If there is a violated constraint add it as a cut
    if violated_constraint_data:
        
        print("!!!!!!!!!!!!!!!!!!!!!!!!!!!! Adding Cut !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")

        # Unpack the tuple to get the data we want
        delta_minus_S, v = violated_constraint_data  

        # Translate the cut into the presolve index
        colind, rowcoef = [], []
        
        drhsp, status = prob.presolverow(      rowtype = "G", # 'G' for >= constraints
                                               origcolind = [prob.getIndex(y[e]) for e in delta_minus_S] + [prob.getIndex(f_i[v])], # Index values of original variables
                                               origrowcoef = [1] * len(delta_minus_S) + [-1], # Coefficients for original variables
                                               origrhs = 0, # Right-hand side of constraint in original variables
                                               maxcoefs=prob.attributes.cols, # what does this do?
                                               colind=colind, rowcoef=rowcoef # where to output the new ones
                                        ) 
    
        # Now we add the translated cut
        prob.addcuts(
                            cuttype=[1],  # General cut
                            rowtype=['G'],  # Presolved row type are we sure??
                            rhs=[drhsp],  # Presolved RHS
                            start=[0, len(colind)],  # Start indices what does this do? 
                            colind=colind,  # Presolved column indices
                            cutcoef=rowcoef  # Presolved coefficients
                    )
        

        print("++++++++++++++++++++++++++++++ Cut Added! ++++++++++++++++++++++++++++++")

    # Continue solving
    return(feas)

In [29]:
# Define the optimization model
model = xp.problem()



# Decision variables
y = {e: xp.var(vartype=xp.binary, name=f"y_{e}") for e in edges}  # Edge selection
z = {c: xp.var(vartype=xp.binary, name=f"z_{c}") for c in all_cycles}  # Cycle selection
f_i = {v: xp.var(vartype=xp.binary) for v in nodes}  # Flow in decision variable
f_o = {v: xp.var(vartype=xp.binary) for v in nodes}  # Flow out decision variable

# Add decision variables
model.addVariable(list(y.values()) + list(z.values())+ list(f_i.values()) + list(f_o.values()))



# Xpress uses indexing when in callback thats why we need to create a dictionary for the ids. I suspect when you run this on actual data
# you would want to do this in another code cell just so that it isn't repeated every time you solve the model for debugging. 
# e.g., id_vars[("y", ('NDD1', 'P1') )] = 0 i.e., the decision variable to connect nodes NDD1 and P1 is the first decision variable in Xpress.

# Initialize id_vars dictionary and counter to assign sequential values
id_vars = {}
counter = 0

# Populate id_vars with ("y", key) tuples
for key in y.keys():
    id_vars[("y", key)] = counter
    counter += 1
    

# Populate id_vars with ("z", key) tuples
for key in z.keys():
    id_vars[("z", key)] = counter
    counter += 1

# Populate id_vars with ("f_i", key) tuples
for key in f_i.keys():
    id_vars[("f_i", key)] = counter
    counter += 1

# Populate id_vars with ("f_o", key) tuples
for key in f_o.keys():
    id_vars[("f_o", key)] = counter
    counter += 1


# Create temporary storage for the callback solutions to be used:
y_temp = {e: 0 for e in edges}
z_temp = {c: 0 for c in all_cycles} 
f_i_temp = {v: 0 for v in nodes}  
f_o_temp = {v: 0 for v in nodes}  


# Define the objective function which is to maximize sum of weights of selected edges and cycles
model.setObjective(xp.Sum(edges[e] * y[e] for e in edges) + xp.Sum(all_cycles[c] * z[c] for c in all_cycles), sense=xp.maximize)






### Constraints:

# 1. Defining f_i and f_o i.e., the incoming and outgoing kidneys to a node v
for v in nodes:
    model.addConstraint(xp.Sum([y[e] for e in edges if e[1] == v]) == f_i[v])
    model.addConstraint(xp.Sum([y[e] for e in edges if e[0] == v]) == f_o[v])

# 2. Ensure that if a node is in an actived cycle, the associated edges that involve it are turned off
for v in pairs:
    model.addConstraint(f_o[v] + xp.Sum([z[c] for c in all_cycles if v in c]) <= f_i[v] + xp.Sum([z[c] for c in all_cycles if v in c]))
    model.addConstraint(f_i[v] + xp.Sum([z[c] for c in all_cycles if v in c]) <= 1)

# 3. Ensure that altruistic donors donate at most one kidney  
for v in altruistic_donors:
    model.addConstraint(f_o[v] <= 1)




# Add the call back function to detect and add violated constraints during LP relaxation
model.addcbpreintsol(separation_cbpreintsol, None, 1) # why do we need to add htis? 
model.addcboptnode(separation_cboptnode, None, 1)




# Solve the model
model.solve()





# Get the current solution
current_sol = ({e: model.getSolution(y[e]) for e in edges}, {c: model.getSolution(z[c]) for c in all_cycles}, {v: model.getSolution(f_i[v]) for v in nodes})

# Extract and print solution
print("Objective Value:", model.getObjVal())
selected_edges = [e for e in edges if model.getSolution(y[e]) > 0.05]
selected_cycles = [c for c in all_cycles if model.getSolution(z[c]) > 0.05]
print("Selected Edges:", selected_edges)
print("Selected Cycles:", selected_cycles)

FICO Xpress v9.4.2, Community, solve started 20:31:10, Feb 20, 2025
Heap usage: 394KB (peak 394KB, 200KB system)
Maximizing MILP noname using up to 14 threads and up to 15GB memory, with these control settings:
OUTPUTLOG = 1
NLPPOSTSOLVE = 1
XSLP_DELETIONCONTROL = 0
XSLP_OBJSENSE = -1
Original problem has:
        15 rows           15 cols           32 elements        15 entities
Presolved problem has:
         7 rows            8 cols           20 elements         8 entities
Presolve finished in 0 seconds
Heap usage: 426KB (peak 447KB, 200KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.00e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.00e+00]
  Objective      [min,max] : [ 1.00e-01,  1.40e+01] / [ 1.00e-01,  1.40e+01]
Autoscaling applied standard scaling

Will try to keep branch and bound tree memory usage below 8.7GB
@@@@@@@@@@@@@@@@@@@@@@@@@@ cbpre